In [1]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181011135730-0001


Scraping the url is next

In [2]:
parsed_table_data = []
myTable = soup.find('table',{'class':'wikitable sortable'})
rows = myTable.findAll('tr')
for row in rows:
    children = row.findChildren(recursive=False)
    row_text = []
    for child in children:
        clean_text = child.text
        #This is to discard reference/citation links
        clean_text = clean_text.split('&#91;')[0]
        #This is to clean the header row of the sort icons
        clean_text = clean_text.split('&#160;')[-1]
        clean_text = clean_text.strip()
        row_text.append(clean_text)
    parsed_table_data.append(row_text)


After cleaning the data, let's take a look at the obtained list:

In [3]:
parsed_table_data[0:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

Now let's convert it to a dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
import pandas as pd
columns_names = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(parsed_table_data,columns = columns_names)
df = df.drop([0])
df.head()

Let's only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [ ]:
df = df[df.Borough != 'Not assigned']
df.head(10)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [ ]:
 df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']

In [ ]:
df.loc[df.Neighbourhood == 'Not assigned',['Neighbourhood']] =  df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']
df.head(10)

If more than one neighborhood can exist in one postal code area, these will be combined into one row with the neighborhoods separated with a comma.

In [ ]:
df = df.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(set(x))).reset_index()

In [ ]:
df.head(10)

In [ ]:
df.shape